In [1]:
# Install necessary libraries if not already installed
!pip install -q transformers datasets tqdm

# Import necessary libraries
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from datasets import load_dataset
import torch
import re
from tqdm import tqdm
import random

# Check if CUDA (GPU) is available, and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the GPT-Neo model and tokenizer, and move the model to the device
model_name = "EleutherAI/gpt-neo-2.7B"
model = GPTNeoForCausalLM.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load the MedQA dataset with training and testing splits
dataset = load_dataset("GBaker/MedQA-USMLE-4-options")
train_dataset = dataset['train']
test_dataset = dataset['test']

Using device: cuda


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [19]:
# Function to create few-shot examples from the training data
def create_few_shot_examples(train_data, num_examples):
    """
    Create a few-shot prompt using `num_examples` samples from the training dataset.
    Each example will include a question, options, and the correct answer.
    """
    # Shuffle and select random examples from the training set for diversity
    shuffled_data = random.sample(list(train_data), len(train_data))
    
    examples = ""
    selected_categories = set()  # Keep track of categories to ensure diversity
    
    # Use tqdm to monitor the progress of few-shot example creation
    for sample in tqdm(shuffled_data, desc="Creating few-shot examples", total=len(train_data)):
        question = sample['question']
        options = sample['options']
        correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

        # Ensure diversity by limiting to different topics or question types (if available)
        if len(selected_categories) >= num_examples:
            break

        # Prepare the few-shot format (Question, Options, Answer)
        options_text = ', '.join([f"{key}: {value}" for key, value in options.items()])
        example = f"Question: {question}\nOptions: {options_text}\nAnswer: {correct_answer}\n\n"
        examples += example

        # Use some logic to track different question types/domains (optional)
        selected_categories.add(question)  # Replace with refined logic for topic detection
    
    return examples

# Function to perform inference with few-shot prompting and match answer using regex
def get_model_prediction(question, options_dict, few_shot_prompt):
    """
    Generate a few-shot prediction using GPT-Neo for a question with multiple choices.
    The options are provided as a dictionary with keys: 'A', 'B', 'C', 'D'.
    The few-shot examples are passed as part of the prompt.
    """
    # Prepare the prompt with few-shot examples + the current question
    options_text = ', '.join([f"{key}: {value}" for key, value in options_dict.items()])
    input_text = f"{few_shot_prompt}\nQuestion: {question}\nOptions: {options_text}\nAnswer:"''

    # print("INPUT" , input_text)
    # print("***********************************************")
    # Tokenize input text
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output (can be tuned using temperature, max_length, etc.)
    output = model.generate(inputs["input_ids"], max_new_tokens=200)

    # Decode the output to text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # print("GEN" , generated_text)
    # print("***********************************************")

    ind=generated_text.find(question)
    extracted_text=generated_text[ind+len(question)+len(options_text)+10:]
    ind2=extracted_text.find("Question")
    predicted=extracted_text[:ind2].strip()
    # print(predicted)

    # Extract the answer after "Answer:" using regex
    # match = re.search(r"Answer:\s*([A-D])", generated_text)
    
    # if match:
    #     predicted_option = match.group(1)
    # else:
    #     predicted_option = None  # Handle case where no match is found
    
    return predicted

# Evaluate model on the test dataset with tqdm to monitor the progress
correct_predictions = 0
total_predictions = 0

# Add tqdm to track the evaluation loop
for sample in tqdm(test_dataset, desc="Evaluating on test dataset", total=len(test_dataset)):
    few_shot_prompt = create_few_shot_examples(train_dataset, num_examples=3)
    question = sample['question']
    options = sample['options']  # This is a dictionary with 'A', 'B', 'C', 'D' as keys
    correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

    # Get the model's prediction using few-shot prompting
    predicted_answer = get_model_prediction(question, options, few_shot_prompt)

    print(predicted_answer)
    print(correct_answer)

    # Check if prediction is correct
    if correct_answer in predicted_answer:
        correct_predictions += 1
    
    total_predictions += 1

    # Calculate and print the accuracy after each iteration
    accuracy = correct_predictions / total_predictions * 100
    print(f"Iteration {total_predictions}: Accuracy so far: {accuracy:.2f}%")
    

# Final accuracy
final_accuracy = correct_predictions / total_predictions * 100
print(f"Final Accuracy: {final_accuracy:.2f}%")


Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 22795.13it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating on test dataset:   0%|          | 1/1273 [00:14<5:07:06, 14.49s/it]

Answer: Disclose the error to the patient and put it in the operative report
Tell the attending that he cannot fail to disclose this mistake
Iteration 1: Accuracy so far: 0.00%



Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 21998.10it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating on test dataset:   0%|          | 2/1273 [00:29<5:11:22, 14.70s/it]

Answer: Inhibition of proteasome
Cross-linking of DNA
Iteration 2: Accuracy so far: 0.00%



Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 24624.09it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating on test dataset:   0%|          | 3/1273 [00:44<5:20:03, 15.12s/it]

...

Answer: Alcoholic beverage prior to bedtime
Stop evening exercise
Iteration 1271: Accuracy so far: 27.14%



Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 21472.55it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating on test dataset: 100%|█████████▉| 1272/1273 [5:26:42<00:15, 15.28s/it]

Answer: Schizotypal personality disorder
Schizotypal personality disorder
Iteration 1272: Accuracy so far: 27.20%



Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 26105.63it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating on test dataset: 100%|██████████| 1273/1273 [5:26:58<00:00, 15.41s/it]

Answer: Asthma
Chronic obstructive pulmonary disease (COPD)
Iteration 1273: Accuracy so far: 27.18%
Final Accuracy: 27.18%
